In [1]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()


2023-05-24 11:44:21.846470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# with a learned mapping, and the tensor basis layer
input_layer = keras.layers.Input(shape=(input_shape),name ='input_layer')
input_tensor_basis = keras.layers.Input(shape=(10,3,3),name='Tensor_input_layer')

#Hidden layer definition
hidden1 = keras.layers.Dense(20,name='Hidden1', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden2)

#The layer of gn, which are coefficients for each of the ten Tn
gn = keras.layers.Dense(10,name='gn', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-4), activation = "selu")(hidden3)

#Multiply the gn by Tn, with the output being the anisotropy tensor
shaped = keras.layers.Reshape((10,1,1),name='Shape_for_dot_product')(gn)
merge = keras.layers.Dot(axes=1, name='Dot_product')([shaped,input_tensor_basis])

#Reshape the output anisotropy tensor, and trim out duplicate values (it is a symmetric matrix). The end result is a 6 component vector.
shaped_output = keras.layers.Reshape((9,1),name='Shaped_output')(merge)
trimmed_output1 = keras.layers.Lambda(lambda x : x[:,0])(shaped_output)
trimmed_output2 = keras.layers.Lambda(lambda x : x[:,1])(shaped_output)
trimmed_output3 = keras.layers.Lambda(lambda x : x[:,2])(shaped_output)
trimmed_output4 = keras.layers.Lambda(lambda x : x[:,4])(shaped_output)
trimmed_output5 = keras.layers.Lambda(lambda x : x[:,5])(shaped_output)
trimmed_output6 = keras.layers.Lambda(lambda x : x[:,8])(shaped_output)
merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output3,trimmed_output4,trimmed_output5,trimmed_output6])


model=keras.Model(inputs=[input_layer, input_tensor_basis], outputs=[merged_output])


optimizer = tf.keras.optimizers.Nadam(learning_rate = 5E-4, clipnorm=1000)
model.compile(optimizer,loss='mse',metrics=['mae', 'mse'])
print(model.summary())

NameError: name 'input_shape' is not defined